In [ ]:
import SimpleITK as sitk

import numpy as np
import os
from ipywidgets import interact, fixed

%matplotlib inline
import matplotlib.pyplot as plt


OUTPUT_DIR = 'output'

image_viewer = sitk.ImageViewer()

# Read DICOM SERIES

In [ ]:
data_directory = os.path.dirname("123/readme.txt")
# Global variable 'selected_series' is updated by the interact function
selected_series = ''
file_reader = sitk.ImageFileReader()
def DICOM_series_dropdown_callback(series_to_load, series_dictionary):
    global selected_series
               # Print some information about the series from the meta-data dictionary
               # DICOM standard part 6, Data Dictionary: http://medical.nema.org/medical/dicom/current/output/pdf/part06.pdf
    file_reader.SetFileName(series_dictionary[series_to_load][0])
    file_reader.ReadImageInformation()
    tags_to_print = {'0010|0010': 'Patient name: ', 
                     '0008|103e': 'Sequence: '}
    
    print('Series_ID: ' + series_to_load)
    for tag in tags_to_print:
        try:
            print(tags_to_print[tag] + file_reader.GetMetaData(tag))
        except: # Ignore if the tag isn't in the dictionary
            pass
    selected_series = series_to_load                    

# Directory contains multiple DICOM studies/series, store
# in dictionary with key being the series ID
series_file_names = {}
series_IDs = sitk.ImageSeriesReader_GetGDCMSeriesIDs(data_directory)
            # Check that we have at least one series


if series_IDs:
    for series in series_IDs:
        series_file_names[series] = sitk.ImageSeriesReader_GetGDCMSeriesFileNames(data_directory, series)
    interact(DICOM_series_dropdown_callback, series_to_load=list(series_IDs), series_dictionary=fixed(series_file_names)); 
else:
    print('Data directory does not contain any DICOM series.')

In [ ]:
data_directory = os.path.dirname("123/readme.txt")
file_reader = sitk.ImageFileReader()

series_file_names = {}
series_IDs = sitk.ImageSeriesReader_GetGDCMSeriesIDs(data_directory)


if series_IDs:
    for series in series_IDs:
        series_file_names[series] = sitk.ImageSeriesReader_GetGDCMSeriesFileNames(data_directory, series)
        
        file_reader.SetFileName(series_file_names[series][0])
        file_reader.ReadImageInformation()
        
        print('Series_ID: ' + series)
        print('Sequence: ' + file_reader.GetMetaData('0008|103e'))
        print('Patient name: ' + file_reader.GetMetaData('0010|0010'))
        
        img = sitk.ReadImage(series_file_names[series])
        print('Size: ' + str(img.GetSize()))
        print('Spacing: ' + str(img.GetSpacing()))
        
        print(' ')

else:
    print('No DICOM series')

In [ ]:
file_reader.SetFileName(series_file_names['1.3.12.2.1107.5.2.30.26129.3.2018062618270667254534358.0.0.0'][0])
file_reader.ReadImageInformation()
file_reader.GetMetaData('0008|103e')
sitk.ReadImage(series_file_names['1.3.12.2.1107.5.2.30.26129.3.2018062618270667254534358.0.0.0'])

In [ ]:
original_image = sitk.ReadImage(series_file_names['1.3.12.2.1107.5.2.30.26129.3.2018062618270667254534358.0.0.0'])
print('origin: ' + str(original_image.GetOrigin()))
print('size: ' + str(original_image.GetSize()))
print('spacing: ' + str(original_image.GetSpacing()))
print('direction: ' + str(original_image.GetDirection()))
print('pixel type: ' + str(original_image.GetPixelIDTypeAsString()))
print('number of pixel components: ' + str(original_image.GetNumberOfComponentsPerPixel()))

# Resample an image to [1 1 1]

In [ ]:
original_image = sitk.ReadImage(sitk.ImageSeriesReader_GetGDCMSeriesFileNames(data_directory,'1.3.12.2.1107.5.2.30.26129.3.2018062208295088450501241.0.0.0'))

In [ ]:
image_viewer.Execute(original_image)

In [ ]:
print('origin: ' + str(original_image.GetOrigin()))
print('size: ' + str(original_image.GetSize()))
print('spacing: ' + str(original_image.GetSpacing()))
print('direction: ' + str(original_image.GetDirection()))
print('pixel type: ' + str(original_image.GetPixelIDTypeAsString()))
print('number of pixel components: ' + str(original_image.GetNumberOfComponentsPerPixel()))

In [ ]:
def resample_image(image):
    resample = sitk.ResampleImageFilter()
    resample.SetInterpolator(sitk.sitkLinear)
    resample.SetOutputDirection(image.GetDirection())
    resample.SetOutputOrigin(image.GetOrigin())
    new_spacing = [1, 1, 1]
    resample.SetOutputSpacing(new_spacing)

    orig_size = np.array(image.GetSize(), dtype=np.int)
    #orig_spacing = original_image.GetSpacing()
    #new_size = orig_size*(orig_spacing/new_spacing)
    #new_size = np.ceil(new_size).astype(np.int) #  Image dimensions are in integers
    #new_size = [int(s) for s in new_size]
   
    resample.SetSize ([240, 240, 155])
    resample.SetOutputOrigin(image.GetOrigin())
    new_image = resample.Execute(image)
    return new_image


In [ ]:
new_image = resample_image(original_image)
print(new_image.GetSize())
print(new_image.GetSpacing())

In [ ]:
image_viewer.Execute(new_image)

# Registration

## Reading file

Contrasted T1 (constant)

In [ ]:
cont_fixed = sitk.ReadImage(series_file_names['1.3.12.2.1107.5.2.30.26129.3.2018062208394729434202541.0.0.0'],sitk.sitkFloat32)
print('size: ' + str(cont_fixed.GetSize()))
print('spacing: ' + str(cont_fixed.GetSpacing()))


Flair register to T1

In [ ]:
flair_moving = sitk.ReadImage(series_file_names['1.3.12.2.1107.5.2.30.26129.3.2018062208295088450501241.0.0.0'],sitk.sitkFloat32)
print('size: ' + str(flair_moving.GetSize()))
print('spacing: ' + str(flair_moving.GetSpacing()))

## Resample and Resize to same parameters

In [ ]:
cont_fixed = resample_image(cont_fixed)
print('size: ' + str(cont_fixed.GetSize()))
print('spacing: ' + str(cont_fixed.GetSpacing()))


In [ ]:
flair_moving = resample_image(flair_moving)
print('size: ' + str(flair_moving.GetSize()))
print('spacing: ' + str(flair_moving.GetSpacing()))

In [ ]:
def registering(fixed, moving):
    R = sitk.ImageRegistrationMethod()
    R.SetMetricAsMattesMutualInformation()
    R.SetInitialTransform(sitk.VersorRigid3DTransform())
    R.SetInterpolator(sitk.sitkLinear)
    #R.SetShrinkFactorsPerLevel(3)
    R.SetOptimizerScalesFromIndexShift()
    
    R.AddCommand( sitk.sitkIterationEvent, lambda: command_iteration(R) )

    outTx = R.Execute(fixed, moving)
    return outTx

In [ ]:
flair_processed = registering(cont_fixed,flair_moving)

In [ ]:
import copy 
fixed = (cont_fixed)
moving = (flair_moving)

R = sitk.ImageRegistrationMethod()

R.SetMetricAsJointHistogramMutualInformation()

R.SetOptimizerAsGradientDescentLineSearch(learningRate=1.0,
                                          numberOfIterations=200,
                                          convergenceMinimumValue=1e-5,
                                          convergenceWindowSize=5)

R.SetInitialTransform(sitk.TranslationTransform(fixed.GetDimension()))

R.SetInterpolator(sitk.sitkLinear)

R.AddCommand( sitk.sitkIterationEvent, lambda: command_iteration(R) )

outTx = R.Execute(fixed, moving)